In [1]:
import json
from google import genai
from google.genai import types
import pathlib
import os
from phase1_subAgent.Generate_Question import generate_question
from phase1_subAgent.Fill_Settings import fill_settings
from phase1_subAgent.Intent_Analysis import intent_analysis
from phase1_subAgent.Generate_Summary import format_session_summary
from phase1_subAgent.Check_Status import Check_Status

In [ ]:
def Execute_Debate_Mode_Setup(lecture_material):

    current_session_profile = {
        "mode_configuration": { 
            "dialogue_style": "Debate_Mode", 
            "interaction_goal": None,        
            "goal_description": None         
        },
        "content_context": { 
            "target_topic": {
                "keyword": None,             
                "description": None          
            },
            "knowledge_boundary": "Lecture_Only" 
        },
        "session_rules": { 
            "max_turns": None,               
            "difficulty_parameter": { 
                "level": None,               
                "custom_constraints": []     
            }
        }
    }

    MAX_STAGNATION = 3
    stagnation_count = 0 
    
    while True:
        
        # 최초 값 채우기 루프
        while True:
            status_result = Check_Status(current_session_profile)

            if status_result.get("status") == "COMPLETE":
                stagnation_count = 0 
                break 
            
            if stagnation_count >= MAX_STAGNATION:
                print("설정을 완료하지 못했습니다. 초기화하거나 종료합니다.")
                return None
            
            # 질문 생성 (반드시 출력은 result.text여야 함)
            question = generate_question(
                missing_fields=status_result.get("missing_fields"), 
                lecture_material=lecture_material
            )
            
            # 값 불러오기
            question = json.loads(question)
            question_text = question.get("question_text")
            question_recommand = question.get("recommendations")

            # 값 출력하기
            print(question_text)

            if question_recommand:
                print(f"추천: {question_recommand}")
            
            # user 입력받기
            user_response = input("사용자: ")
            

            prev_profile_str = str(current_session_profile)

            update_result = fill_settings(current_session_profile, user_response)

            # 값 불러오기
            update_result = json.loads(update_result)
            
            # false가 나온경우
            if not update_result.get("is_valid"):
                stagnation_count += 1
                print("이해하지 못했습니다. 다시 말씀해 주세요.")
                continue
            
            # 업데이트된 profile 가져오기
            new_profile = update_result.get("new_profile")
            
            # true가 나왔는데 업데이트 된게 없는 경우
            if str(new_profile) == prev_profile_str:
                stagnation_count += 1
                print("설정이 변경되지 않았습니다. 구체적으로 말씀해 주세요.")
                continue
            
            # 정상적으로 업데이트 된 경우
            stagnation_count = 0
            current_session_profile = new_profile


        while True:
            summary = format_session_summary(current_session_profile)
            
            print(f"이 설정으로 토론을 시작하시겠습니까?\n{summary}")
            user_response = input("사용자(승인/수정): ")

            intent_analysis_result = intent_analysis(user_response)
            
            # 값 불러오기
            intent_analysis_result = json.loads(intent_analysis_result)
            intent = intent_analysis_result.get("intent")

            if intent == "APPROVE":
                final_check = Check_Status(current_session_profile)
                
                if final_check.get("status") == "INCOMPLETE":
                    print("수정 과정에서 필수 정보가 누락되었습니다. 다시 입력 단계로 돌아갑니다.")
                    break # 최초 루프로 되돌아감
                
                return current_session_profile

            elif intent == "MODIFY":
                update_result = fill_settings(current_session_profile, intent_analysis_result.get("Modifications"))
                
                # 값 불러오기
                update_result = json.loads(update_result)

                if update_result.get("is_valid"):
                    current_session_profile = update_result.get("new_profile")
                    print("설정이 수정되었습니다.")
                else:
                    print("수정 사항을 명확히 인식하지 못했습니다.")
                continue

            else:
                print("다시 말씀해 주세요. (예: '좋아', '어디를 수정해줘')")



In [3]:
# 테스트 실행
lecture_material = "/Users/jhkim/Desktop/Debate_Agent/KOSPI.pdf"
session_profile = Execute_Debate_Mode_Setup(lecture_material)

2026년 코스피 지수의 5,250 상향 조정과 리레이팅(Re-rating) 가능성에 대한 흥미로운 리포트입니다. 이 자료를 바탕으로 어떤 목표를 가지고 토론을 진행해볼까요? 집중하고 싶은 주제와 대화의 깊이, 그리고 진행할 턴 수도 함께 선택해 주세요.
추천: {'mode_configuration.interaction_goal': ['Concept_Learning (개념 학습형: 교과서적 개념 검증 및 학습)', 'Critical_Critique (심화 탐구 및 비판형: 이론의 한계점 비판 및 심층 토론)'], 'mode_configuration.goal_description': ['코스피 리레이팅의 핵심 동력과 밸류에이션 산출 근거 파악', '글로벌 경기 둔화 우려와 수출 증가율 전망 사이의 괴리 분석', 'AI 인프라 및 국방 산업 중심의 유동성 집중 현상 비판적 검토'], 'content_context.target_topic.keyword': ['코스피 5,250 상향과 리레이팅', 'AI 인프라 및 전력기기 주도주 전망', '글로벌 확장 재정과 한국 증시 영향'], 'session_rules.max_turns': ['5', '10', '15'], 'session_rules.difficulty_parameter.level': ['Low (시장 지표 및 기초 용어 위주)', 'Medium (매크로 환경 및 밸류에이션 분석 포함)', 'High (전문적인 투자 전략 및 리스크 비평 수준)']}
이 설정으로 토론을 시작하시겠습니까?

[Session Configuration Summary]

1. Mode Configuration
   - Dialogue Style: Debate_Mode
   - Interaction Goal: Critical_Critique
   - Description: 기존 이론의 한계점이나 개선안을 사용자가 제시하고, 상대 에이전트가 이에 대해 강하게 비판하며 논리를 검증하는 방식

2. Content Context
   - Targ

In [5]:
print(type(session_profile))

<class 'dict'>
